# Kaggle Competition
## Titanic - Machine Learning from Disaster

#### titanic_dataset.py 분석 리포트

In [2]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader, random_split

In [3]:
class TitanicDataset(Dataset):  # PyTorch의 Dataset 클래스를 상속하는 TitanicDataset 클래스를 정의
  def __init__(self, X, y):
    # 클래스가 생성될 때, X는 입력 데이터(피처), y는 타겟 데이터(레이블)로 입력받음
    # torch.FloatTensor로 X를 변환하여 저장하고, torch.LongTensor로 y를 변환하여 저장
    # 이 변환은 PyTorch가 텐서 형태로 데이터를 처리할 수 있게 하는 것
    self.X = torch.FloatTensor(X)
    self.y = torch.LongTensor(y)

  def __len__(self):
    # 데이터셋의 길이를 반환하는 함수
    # 이를 통해 데이터 로더가 몇 개의 샘플을 가져올지 알 수 있음
    return len(self.X)

  def __getitem__(self, idx):
    # 데이터셋에서 특정 인덱스에 해당하는 샘플을 반환하는 함수
    # feature는 X에서 idx번째 값을, target은 y에서 idx번째 값을 의미
    # 반환값은 딕셔너리로, 'input'이란 키로 피처를, 'target'이란 키로 타겟을 반환
    feature = self.X[idx]
    target = self.y[idx]
    return {'input': feature, 'target': target}

  def __str__(self):
    # 데이터셋의 기본 정보를 문자열 형태로 반환하는 함수
    # 데이터의 크기, 입력의 모양, 타겟의 모양을 포함
    str = "Data Size: {0}, Input Shape: {1}, Target Shape: {2}".format(
      len(self.X), self.X.shape, self.y.shape
    )
    return str

In [4]:
class TitanicTestDataset(Dataset):  # PyTorch의 Dataset 클래스를 상속하는 TitanicTestDataset 클래스를 정의
  def __init__(self, X):
    # 클래스가 생성될 때, X는 테스트 데이터(피처)로 입력받음
    # torch.FloatTensor로 X를 변환하여 저장
    self.X = torch.FloatTensor(X)

  def __len__(self):
    # 데이터셋의 길이를 반환하는 함수
    # 이를 통해 데이터 로더가 몇 개의 샘플을 가져올지 알 수 있음
    return len(self.X)

  def __getitem__(self, idx):
    # 데이터셋에서 특정 인덱스에 해당하는 샘플을 반환하는 함수
    # feature는 X에서 idx번째 값을 의미
    # 반환값은 딕셔너리로, 'input'이란 키로 피처를 반환
    feature = self.X[idx]
    return {'input': feature}

  def __str__(self):
    # 데이터셋의 기본 정보를 문자열 형태로 반환하는 함수
    # 데이터의 크기와 입력 데이터의 모양을 포함
    str = "Data Size: {0}, Input Shape: {1}".format(
      len(self.X), self.X.shape
    )
    return str

In [5]:
def get_preprocessed_dataset():
    # 데이터 경로 설정
    CURRENT_FILE_PATH = '../../../_02_homeworks/homework_2/'  # 데이터가 저장된 실제 경로로 변경해야 함

    # 학습 데이터(train.csv)와 테스트 데이터(test.csv)의 경로를 설정
    train_data_path = os.path.join(CURRENT_FILE_PATH, "train.csv")
    test_data_path = os.path.join(CURRENT_FILE_PATH, "test.csv")

    # 학습 데이터와 테스트 데이터를 각각 읽어 DataFrame으로 로드
    train_df = pd.read_csv(train_data_path)
    test_df = pd.read_csv(test_data_path)

    # 학습 데이터와 테스트 데이터를 하나의 DataFrame으로 결합 (병합)
    all_df = pd.concat([train_df, test_df], sort=False)

    # 여러 전처리 함수들을 순차적으로 호출하여 데이터를 전처리
    # get_preprocessed_dataset_1부터 get_preprocessed_dataset_6까지 전처리 과정을 거침
    all_df = get_preprocessed_dataset_1(all_df)
    all_df = get_preprocessed_dataset_2(all_df)
    all_df = get_preprocessed_dataset_3(all_df)
    all_df = get_preprocessed_dataset_4(all_df)
    all_df = get_preprocessed_dataset_5(all_df)
    all_df = get_preprocessed_dataset_6(all_df)

    # Survived 열이 비어있지 않은 데이터는 학습용 데이터셋, 비어있는 데이터는 테스트용 데이터셋으로 분리
    # train_X는 피처(입력 데이터), train_y는 타겟(정답 레이블)로 설정
    train_X = all_df[~all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True)
    train_y = train_df["Survived"]

    # 테스트 데이터는 Survived 열이 비어있는 부분을 사용 (테스트 데이터의 타겟은 없음)
    test_X = all_df[all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True)

    # TitanicDataset 클래스를 이용하여 학습 데이터셋을 생성
    dataset = TitanicDataset(train_X.values, train_y.values)
    
    # 학습 데이터셋을 80%는 훈련용, 20%는 검증용으로 분할
    train_dataset, validation_dataset = random_split(dataset, [0.8, 0.2])

    # TitanicTestDataset 클래스를 이용하여 테스트 데이터셋을 생성
    test_dataset = TitanicTestDataset(test_X.values)

    # 훈련 데이터셋, 검증 데이터셋, 테스트 데이터셋을 반환
    return train_dataset, validation_dataset, test_dataset


In [6]:
def get_preprocessed_dataset_1(all_df):
    # Pclass별로 Fare(요금)의 평균값을 계산하여 결측치를 채우는 작업

    # Pclass별로 그룹화하여 Fare의 평균값을 계산
    Fare_mean = all_df[["Pclass", "Fare"]].groupby("Pclass").mean().reset_index()
    
    # 계산된 평균값을 담은 열 이름을 Pclass와 Fare_mean으로 설정
    Fare_mean.columns = ["Pclass", "Fare_mean"]
    
    # 원본 데이터프레임 all_df와 계산된 평균값을 병합
    # 병합 기준은 Pclass이며, Pclass별 평균 Fare를 함께 연결
    all_df = pd.merge(all_df, Fare_mean, on="Pclass", how="left")
    
    # Fare 값이 결측치인 경우, Pclass에 해당하는 평균 Fare 값으로 대체
    all_df.loc[(all_df["Fare"].isnull()), "Fare"] = all_df["Fare_mean"]

    # 추가된 Fare_mean 열을 포함한 데이터프레임을 반환
    return all_df

In [7]:
def get_preprocessed_dataset_2(all_df):
    # Name 열을 가족 이름, 호칭, 개인 이름 세 개의 컬럼으로 분리하여 처리

    # Name 열을 쉼표(,)와 마침표(.)를 기준으로 분리
    # n=2: 쉼표와 마침표로 최대 두 번 분리 (즉, 세 개의 부분으로 나뉨)
    # expand=True: 결과를 데이터프레임으로 확장
    name_df = all_df["Name"].str.split("[,.]", n=2, expand=True)

    # 분리된 세 개의 열 이름을 family_name(가족 이름), honorific(호칭), name(이름)으로 설정
    name_df.columns = ["family_name", "honorific", "name"]

    # 각 열의 문자열에서 불필요한 공백을 제거
    name_df["family_name"] = name_df["family_name"].str.strip()  # 가족 이름 공백 제거
    name_df["honorific"] = name_df["honorific"].str.strip()      # 호칭 공백 제거
    name_df["name"] = name_df["name"].str.strip()                # 개인 이름 공백 제거

    # 원래의 데이터프레임 all_df와 분리된 name_df를 열 단위로 병합
    all_df = pd.concat([all_df, name_df], axis=1)

    # 처리된 데이터프레임을 반환
    return all_df

In [8]:
def get_preprocessed_dataset_3(all_df):
    # honorific(호칭)별로 Age(나이)의 중앙값을 계산하여 결측치를 채우는 작업

    # honorific별로 그룹화하여 Age의 중앙값을 계산하고 반올림
    # 중앙값을 사용하는 이유는 극단값에 덜 민감하기 때문 (평균보다 대표값으로 적합)
    honorific_age_mean = all_df[["honorific", "Age"]].groupby("honorific").median().round().reset_index()

    # 계산된 중앙값 열의 이름을 "honorific_age_mean"으로 설정
    honorific_age_mean.columns = ["honorific", "honorific_age_mean"]

    # 원본 데이터프레임 all_df와 계산된 중앙값을 honorific을 기준으로 병합
    all_df = pd.merge(all_df, honorific_age_mean, on="honorific", how="left")

    # Age 값이 결측치인 경우, 해당 honorific의 중앙값으로 결측치를 채움
    all_df.loc[(all_df["Age"].isnull()), "Age"] = all_df["honorific_age_mean"]

    # 더 이상 필요 없는 "honorific_age_mean" 열을 삭제
    all_df = all_df.drop(["honorific_age_mean"], axis=1)

    # 결측치가 처리된 데이터프레임을 반환
    return all_df


In [9]:
def get_preprocessed_dataset_4(all_df):
    # 가족 수(family_num) 컬럼을 새롭게 추가 (Parch와 SibSp의 합)
    # Parch: 부모 또는 자녀의 수, SibSp: 형제자매 또는 배우자의 수
    # 두 값을 더해 해당 승객의 가족 수를 계산
    all_df["family_num"] = all_df["Parch"] + all_df["SibSp"]

    # 혼자 탑승 여부(alone) 컬럼을 새롭게 추가
    # family_num이 0이면 혼자 탑승한 것이므로 "alone" 값을 1로 설정
    all_df.loc[all_df["family_num"] == 0, "alone"] = 1

    # family_num이 0이 아닌 경우에는 혼자가 아니므로 "alone" 값을 0으로 설정
    all_df["alone"].fillna(0, inplace=True)

    # 학습에 불필요한 컬럼을 제거
    # PassengerId, Name, family_name, name, Ticket, Cabin은 모델 학습에 의미가 없으므로 삭제
    all_df = all_df.drop(["PassengerId", "Name", "family_name", "name", "Ticket", "Cabin"], axis=1)

    # 전처리된 데이터프레임 반환
    return all_df


In [10]:
def get_preprocessed_dataset_5(all_df):
    # honorific(호칭) 값을 줄이기 위해 주요한 호칭만 남기고 나머지는 'other'로 통합

    # honorific 값이 "Mr", "Miss", "Mrs", "Master"가 아닌 경우는 모두 "other"로 변경
    all_df.loc[
    ~(
            (all_df["honorific"] == "Mr") |
            (all_df["honorific"] == "Miss") |
            (all_df["honorific"] == "Mrs") |
            (all_df["honorific"] == "Master")
    ),
    "honorific"
    ] = "other"

    # Embarked(탑승 항구) 결측값을 "missing"으로 채움
    all_df["Embarked"].fillna("missing", inplace=True)

    # 전처리된 데이터프레임 반환
    return all_df

In [11]:
def get_preprocessed_dataset_6(all_df):
    # 카테고리형 변수들을 LabelEncoder를 사용해 수치형 값으로 변환

    # 데이터프레임에서 데이터 타입이 object(문자형)인 컬럼을 추출하여 카테고리 피처로 설정
    category_features = all_df.columns[all_df.dtypes == "object"]

    # LabelEncoder를 sklearn에서 불러옴
    from sklearn.preprocessing import LabelEncoder

    # 각 카테고리형 피처에 대해 반복하면서 LabelEncoder로 수치형 값으로 변환
    for category_feature in category_features:
        le = LabelEncoder()  # LabelEncoder 객체 생성
        if all_df[category_feature].dtypes == "object":  # 피처가 object 타입인지 확인
            le = le.fit(all_df[category_feature])  # 해당 피처에 대해 LabelEncoder 학습
            all_df[category_feature] = le.transform(all_df[category_feature])  # 변환하여 수치형 값으로 대체

    # 변환된 데이터프레임 반환
    return all_df

In [12]:
from torch import nn

# MyModel 클래스는 PyTorch의 nn.Module을 상속받아 정의된 모델 클래스
class MyModel(nn.Module):
    def __init__(self, n_input, n_output):
        # 부모 클래스(nn.Module)의 초기화 함수를 호출
        super().__init__()

        # nn.Sequential을 사용하여 모델의 레이어를 순차적으로 쌓음
        self.model = nn.Sequential(
            # 첫 번째 Linear 레이어: 입력 크기는 n_input, 출력 크기는 30
            nn.Linear(n_input, 30),
            # 활성화 함수로 ReLU를 사용
            nn.ReLU(),
            # 두 번째 Linear 레이어: 입력 크기는 30, 출력 크기는 30
            nn.Linear(30, 30),
            # 다시 ReLU 활성화 함수
            nn.ReLU(),
            # 세 번째 Linear 레이어: 입력 크기는 30, 출력 크기는 n_output
            nn.Linear(30, n_output),
        )

    # forward 함수는 입력 데이터를 받아 모델을 통과시킨 후 출력을 반환
    def forward(self, x):
        # 입력 데이터를 정의된 모델에 통과시킴
        x = self.model(x)
        # 최종 결과 반환
        return x


In [13]:
def test(test_data_loader):
    # 테스트 함수 시작 표시 출력
    print("[TEST]")
    
    # 테스트 데이터 로더에서 첫 번째 배치(batch)를 가져옴
    batch = next(iter(test_data_loader))
    
    # 배치에서 가져온 입력 데이터의 크기를 출력
    print("{0}".format(batch['input'].shape))
    
    # MyModel 객체를 생성, 입력 차원은 11, 출력 차원은 2 (분류 문제로 추정)
    my_model = MyModel(n_input=11, n_output=2)
    
    # 모델에 입력 데이터를 통과시켜 예측 결과를 얻음
    output_batch = my_model(batch['input'])
    
    # 예측된 결과 중 가장 높은 값을 가진 클래스를 선택 (dim=1은 클래스 차원을 기준으로 최대값을 선택)
    prediction_batch = torch.argmax(output_batch, dim=1)
    
    # 예측된 결과를 출력
    # 892부터 시작하는 인덱스를 사용하여 각 예측 값에 대한 인덱스와 결과를 출력
    for idx, prediction in enumerate(prediction_batch, start=892):
        print(idx, prediction.item())  # idx는 892부터 시작하며, 각 예측 값은 prediction.item()을 사용해 출력


In [14]:
if __name__ == "__main__":
    # 메인 함수 시작. 여기서 데이터셋을 불러오고, 데이터 로더를 설정하며, 훈련 및 테스트 실행

    # 전처리된 데이터셋을 불러옴 (훈련, 검증, 테스트 데이터셋)
    train_dataset, validation_dataset, test_dataset = get_preprocessed_dataset()

    # 데이터셋의 크기를 출력 (훈련, 검증, 테스트 데이터셋의 샘플 수)
    print("train_dataset: {0}, validation_dataset.shape: {1}, test_dataset: {2}".format(
        len(train_dataset), len(validation_dataset), len(test_dataset)
    ))
    
    # 구분선을 출력하여 시각적으로 구분
    print("#" * 50, 1)

    # 훈련 데이터셋의 각 샘플을 출력 (입력 값과 타겟 값)
    for idx, sample in enumerate(train_dataset):
        print("{0} - {1}: {2}".format(idx, sample['input'], sample['target']))

    # 다시 구분선을 출력
    print("#" * 50, 2)

    # DataLoader 객체를 생성하여 훈련, 검증, 테스트 데이터를 배치 단위로 불러올 수 있게 설정
    # 훈련 데이터는 16개씩 묶어서 무작위로 셔플
    train_data_loader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)
    # 검증 데이터도 16개씩 묶어서 셔플
    validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=16, shuffle=True)
    # 테스트 데이터는 데이터셋 전체 크기만큼 한 번에 로드 (셔플하지 않음)
    test_data_loader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset))

    # 훈련 데이터 배치 정보 출력
    print("[TRAIN]")
    for idx, batch in enumerate(train_data_loader):
        # 각 배치의 입력과 타겟 크기를 출력
        print("{0} - {1}: {2}".format(idx, batch['input'].shape, batch['target'].shape))

    # 검증 데이터 배치 정보 출력
    print("[VALIDATION]")
    for idx, batch in enumerate(validation_data_loader):
        # 각 배치의 입력과 타겟 크기를 출력
        print("{0} - {1}: {2}".format(idx, batch['input'].shape, batch['target'].shape))

    # 마지막 구분선을 출력
    print("#" * 50, 3)

    # 테스트 함수 호출. 테스트 데이터 로더를 인자로 전달하여 테스트 실행
    test(test_data_loader)


/tmp/ipykernel_1090505/1166421150.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  all_df["alone"].fillna(0, inplace=True)
/tmp/ipykernel_1090505/4090260951.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try u

train_dataset: 713, validation_dataset.shape: 178, test_dataset: 418
################################################## 1
0 - tensor([ 1.0000,  0.0000, 54.0000,  1.0000,  0.0000, 59.4000,  0.0000, 87.5090,
         3.0000,  1.0000,  0.0000]): 1
1 - tensor([ 2.0000,  0.0000, 29.0000,  0.0000,  0.0000, 10.5000,  2.0000, 21.1792,
         3.0000,  0.0000,  1.0000]): 1
2 - tensor([ 3.0000,  0.0000, 22.0000,  2.0000,  0.0000, 23.2500,  1.0000, 13.3029,
         1.0000,  2.0000,  0.0000]): 1
3 - tensor([ 3.0000,  1.0000, 17.0000,  1.0000,  1.0000,  7.2292,  0.0000, 13.3029,
         2.0000,  2.0000,  0.0000]): 0
4 - tensor([ 3.0000,  1.0000, 29.0000,  0.0000,  0.0000,  9.5000,  2.0000, 13.3029,
         2.0000,  0.0000,  1.0000]): 1
5 - tensor([ 3.0000,  0.0000, 15.0000,  0.0000,  0.0000,  7.2250,  0.0000, 13.3029,
         1.0000,  0.0000,  1.0000]): 1
6 - tensor([ 1.0000,  0.0000, 62.0000,  0.0000,  0.0000, 80.0000,  3.0000, 87.5090,
         3.0000,  0.0000,  1.0000]): 1
7 - tensor([ 1.00